# Paradigmas y Programación para Ciencia de Datos
## Programa 6


Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

Carlo Kiliano Ferrera Guadarrama, Aldo Sebastián Altamirano Vázquez, Edgar Omar Cruz Guitierrez, Leonardo Negrete Mancera

In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

def calcular_proba_previa(y_train):
    """
    Calcula el logaritmo de las probabilidades previas de cada clase.
    """

    # Contamos cuántas clases únicas hay.
    n_clases = np.max(y_train) + 1

    # Obtenemos el número total de documentos
    n_documentos = len(y_train)

    # Vector para guardar los priors
    log_previas = np.zeros(n_clases)

    for k in range(n_clases):
        #N_k = Contamos cuántos documentos pertenecen a la clase k
        n_k = np.sum(y_train == k)

        # Calculamos la probabilidad N_k/N_total
        epsilon = 1e-10
        previa_k = (n_k + epsilon) / (n_documentos + epsilon * n_clases)

        # Calculamos y guardamos la probabilidad previa aplicando log
        log_previas[k] = np.log(previa_k)

    return log_previas

In [ ]:
def calcular_prob_pi(X_train, y_train, alpha=1.0):
    """
    Calcula el logaritmo de las probabilidades condicionales (matriz PI)
    """

    #Obtenemos las dimensiones
    n_clases = np.max(y_train) + 1
    #sacamos el número de palabras que se usan
    n_features = X_train.shape[1]

    # Inicializamos matrices para los conteos
    feature_counts_per_class = np.zeros((n_clases, n_features))
    total_counts_per_class = np.zeros(n_clases)

    # Calculamos los conteos
    for k in range(n_clases):
        #Obtenemos todos los documentos que pertenecen a la clase k
        X_k = X_train[y_train == k]

        #Sumamos las columnas para obtener el total de cada feature j en la clase k.
        feature_counts_per_class[k, :] = np.sum(X_k, axis=0) + alpha

        #Sumamos todos esos conteos para obtener el total depalabras en la clase k.
        total_counts_per_class[k] = np.sum(feature_counts_per_class[k, :])

    log_numerador = np.log(feature_counts_per_class)
    log_denominador = np.log(total_counts_per_class[:, np.newaxis])

    #Calculamos log(π_k,j)
    log_pi = log_numerador - log_denominador

    return log_pi

In [ ]:
import numpy as np

def clasificador_bayesiano_multinomial(X, LOG_PI, LOG_PRIORS):
    """
    Clasifica documentos usando Naive Bayes Multinomial en su forma logarítmica.
    """

    # Número de documentos (filas en X)
    n_documentos = X.shape[0]
    # Número de clases (filas en LOG_PI)
    n_clases = LOG_PI.shape[0]

    # Arreglo para guardar las predicciones (una por documento)
    predicciones = []

    #Iteramos sobre cada documento k
    for k in range(n_documentos):

        # Obtenemos el vector de conteo para este documento
        x_doc = X[k, :]

        #Guarda el puntaje log-posterior de este documento para cada clase
        puntajes_clase = np.zeros(n_clases)

        #Iteramos sobre cada clase k
        for i in range(n_clases):

            #Empezamos con el log-prior de la clase k
            log_posterior = LOG_PRIORS[i]

            #Calculamos la suma de log-verosimilitud
            log_verosimilitud_suma = np.dot(x_doc, LOG_PI[i, :])

            #Sumamos el log-prior y la log-verosimilitud
            log_posterior += log_verosimilitud_suma

            # Guardamos el puntaje final para esta clase
            puntajes_clase[i] = log_posterior


        # encontramos la clase con el puntaje más alto
        predicciones.append(np.argmax(puntajes_clase))

    return np.array(predicciones)



In [ ]:
#Hacemos una prueba

X_train = np.array([
    [2, 1, 0, 0, 0],  # Doc 0: "ver ver partido" (Clase 0)
    [1, 2, 0, 0, 0],  # Doc 1: "ver partido partido" (Clase 0)
    [0, 0, 3, 1, 2],  # Doc 2: "IA IA IA algoritmo python python" (Clase 1)
    [0, 0, 1, 2, 1]   # Doc 3: "IA algoritmo algoritmo python" (Clase 1)
])
y_train = np.array([0, 0, 1, 1]) #0=Deportes, 1=Programación
X_test = np.array([
    [1, 0, 1, 1, 0]  # "ver algoritmo IA"
])

ALPHA = 1.0



In [ ]:
#Probamos nuestra implementación
print("Implementación desde cero")
#Lo primero que hacemos es calcular las probabilidades
prob_previas= calcular_proba_previa(y_train)
prob_pi = calcular_prob_pi(X_train, y_train, alpha=ALPHA)

#Ahora predecimos
pred_custom = clasificador_bayesiano_multinomial(X_test, prob_pi, prob_previas)

print(f"Log Priors (propio):\n{prob_previas}")
print(f"\nLog Pi (propio):\n{prob_pi}")
print(f"\nPredicción (propio): {pred_custom}")

Implementación desde cero
Log Priors (propio):
[-0.69314718 -0.69314718]

Log Pi (propio):
[[-1.01160091 -1.01160091 -2.39789527 -2.39789527 -2.39789527]
 [-2.7080502  -2.7080502  -1.09861229 -1.32175584 -1.32175584]]

Predicción (propio): [1]


In [ ]:
print("Implementación con SKlearn")
#Entrenamos
model_sklearn = MultinomialNB(alpha=ALPHA, fit_prior=True)
model_sklearn.fit(X_train, y_train)

#Predecimos
pred_sklearn = model_sklearn.predict(X_test)

# Obtenemos las probabilidades calculadas por sklearn
log_priors_sklearn = model_sklearn.class_log_prior_
log_pi_sklearn = model_sklearn.feature_log_prob_

print(f"Log Priors (Sklearn):\n{log_priors_sklearn}")
print(f"\nLog Pi (Sklearn):\n{log_pi_sklearn}")
print(f"\nPredicción (Sklearn): {pred_sklearn}")



# np.allclose se usa para comparar arrays de números flotantes
priors_match = np.allclose(prob_previas, log_priors_sklearn)
pi_match = np.allclose(prob_pi, log_pi_sklearn)
preds_match = np.array_equal(pred_custom, pred_sklearn)


Implementación con SKlearn
Log Priors (Sklearn):
[-0.69314718 -0.69314718]

Log Pi (Sklearn):
[[-1.01160091 -1.01160091 -2.39789527 -2.39789527 -2.39789527]
 [-2.7080502  -2.7080502  -1.09861229 -1.32175584 -1.32175584]]

Predicción (Sklearn): [1]
